In [1]:
import pandas as pd
from glob import glob
import numpy as np

In [2]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [3]:
def prepare_data(pattern):
    data=pd.DataFrame.from_records(pd.concat([pd.read_json(file,orient="records") for file in glob(pattern)]).task.tolist())
    data_task=pd.DataFrame.from_records(pd.concat([pd.read_json(file,orient="records") for file in glob(pattern)]).taskRuns.apply(lambda x: x[0] if x else {}).tolist())
    answers = pd.DataFrame.from_records(data_task.answer.fillna(0).apply( lambda answer: { entry['name']:entry['value'] for entry in answer} if answer else {}))
    answers.columns=answers.columns+"_ans"
    data_merged = pd.concat([data,data_task,answers],axis=1) 
    return data_merged

In [4]:
start_data = prepare_data("old/*.json")
ebates_data = prepare_data("ebates/*.json")

In [5]:
columns = start_data.columns.intersection(ebates_data.columns).unique()
all_merged = pd.concat([start_data[columns],ebates_data.ix[:,columns]])
all_merged = all_merged.sort_values(by=['productUrl','finish_time'],ascending=False)
all_merged.drop_duplicates(subset="productUrl",keep="first",inplace=True)
all_merged["TopLevel"]=all_merged.categoryNamePath.apply(lambda x: x.split(">")[0])